In [36]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [37]:
session = requests.Session()
login_url = 'https://onlinejudge.org/index.php?option=com_comprofiler&task=login'
response = session.get(login_url)
soup = BeautifulSoup(response.text, 'html.parser')

In [38]:
form = soup.find('form', id='mod_loginform')
hidden_inputs = form.find_all('input', type='hidden')
form_data = {input.get('name'): input.get('value') for input in hidden_inputs}

In [39]:
form_data['username'] = 'ToxicArcher'
form_data['passwd'] = '123456789'
form_data['remember'] = 'yes'
form_data['Submit'] = 'Login'

In [40]:
login_response = session.post(form['action'], data=form_data)
assert 'Logout' in login_response.text, 'Login failed!'

In [41]:
category_url = 'https://onlinejudge.org/index.php?option=com_onlinejudge&Itemid=8&category=606'
response = session.get(category_url)
soup = BeautifulSoup(response.text, 'html.parser')

categories = {
    'Super Easy': '607',
    'Easy': '608',
    'Medium': '609'
}

category_links = {name: f"https://onlinejudge.org/index.php?option=com_onlinejudge&Itemid=8&category={id}" for name, id in categories.items()}


In [52]:
problem_data = {}

for category, link in category_links.items():
    response = session.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    problems = soup.find_all('a', href=True)
    
    print(f'\n{category}:')
    problem_data[category] = []
    for a in problems:
        if 'problem' in a['href']:
            problem_url = f"https://onlinejudge.org{a['href']}"
            problem_name_full = a.text.strip()
            problem_name = problem_name_full.split(' - ')[1]
            print(problem_name)
            problem_data[category].append((problem_name, problem_url))


Super Easy:
TEX Quotes
Celebrity jeopardy
Combination Lock
Searching for Nessy
Relational Operator
Parking
Division of Nlogonia
Automatic Answer
Cost Cutting
Language Detection
Emoogle Balance
One-Two-Three
Packing for Holiday
Save Setu
Hajj-e-Akbar

Easy:
Secret Research
Loansome Car Buyer
Ecological Premium
The Swallowing Ground
Summing Digits
Event Planning
Sub-prime
Jumping Mario
Horror Dash
Lumberjack Sequencing
Google is Feeling Lucky
Tariff Plan
Zapping
Robot Instructions
A Special "Happy Birthday" Song!!!

Medium:
Greedy Gift Givers
The Snail
Blowing Fuses
Request for Proposal
Zeros and Ones
Love Calculator
Prerequisites?
Bender B. RodrÃ­guez Problem
Train Tracks
Burger Time?
Laser Sculpture
Digits
Brainfuck
Hardest Problem Ever (Easy)


In [ ]:
base_folder = 'Getting Started: The Easy Problems'

for category, problems in problem_data.items():
    category_folder = os.path.join(base_folder, category)
    for problem_name, problem_link in problems:
        problem_id = problem_link.split('=')[-1]  # Assuming the ID is at the end of the URL
        solution_file = os.path.join(category_folder, f"{problem_name}.py")  # Adjust file extension if necessary
        
        if os.path.exists(solution_file):
            with open(solution_file, 'r') as file:
                solution_code = file.read()
                
            problem_response = session.get(problem_link)
            problem_soup = BeautifulSoup(problem_response.text, 'html.parser')
            
            form = problem_soup.find('form', {'id': 'problem_submission_form'})
            form_data = {input.get('name'): input.get('value') for input in form.find_all('input', type='hidden')}
            
            form_data['code'] = solution_code
            form_data['language'] = '5'  # Adjust based on actual language code
            
            submission_url = form['action']
            submission_response = session.post(submission_url, data=form_data)
            
            if 'Submission received' in submission_response.text:
                print(f"Successfully submitted solution for problem {problem_name}")
            else:
                print(f"Failed to submit solution for problem {problem_name}")
